In [1]:
from transformers import AutoTokenizer

# Load a tokenizer (example: GPT-2 tokenizer)
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/pythia-1.4b-deduped")

def count_hf_tokens(text):
    tokens = tokenizer.tokenize(text)
    print(tokens)
    return len(tokens)

# example = "Tails"
# print("Number of tokens:", count_hf_tokens(example))
decoded_text = tokenizer.decode(37465)
print(decoded_text)

tails


In [8]:
import random
import csv
import os


ALL_NAMES = [
    "Alice", "Bob", "Charlie", "Dana", "Eve", "Frank", "Grace", "Hank", "Ivy", "Jack",
    "Kara", "Liam", "Mona", "Nate", "Olive", "Paul", "Quinn", "Rita", "Sam", "Tina",
    "Uma", "Vince", "Wade", "Xena", "Yara", "Zane", "Ava", "Ben", "Cleo", "Drew",
    "Ella", "Finn", "Gina", "Hugo", "Isla", "Jon", "Kiki", "Leo", "Mira", "Noah",
    "Omar", "Pia", "Qin", "Ray", "Sara", "Tom", "Uli", "Vera", "Will", "Zoe",
    "Adrian", "Bianca", "Caleb", "Daphne", "Elijah", "Freya", "Gavin", "Hazel", "Iris", "Jasper",
    "Kendall", "Lola", "Marcus", "Nina", "Orion", "Phoebe", "Quentin", "Rosa", "Silas", "Talia",
    "Ulric", "Valerie", "Wes", "Xiomara", "Yusuf", "Zelda", "Amber", "Blake", "Carmen", "Derek",
    "Esme", "Felix", "Georgia", "Heidi", "Isaac", "Jada", "Kai", "Lena", "Milo", "Noelle",
    "Otis", "Poppy", "Quade", "Ronan", "Sienna", "Theo", "Umair", "Violet", "Willa", "Xander",
    "Yasmine", "Zion", "Asher", "Brielle", "Cyrus", "Delia", "Elias", "Fiona", "Griffin", "Holly",
    "Indira", "Joel", "Kira", "Luca", "Maya", "Nico", "Odelia", "Parker", "Qiana", "Rafael",
    "Sage", "Tristan", "Ursula", "Viktor", "Wynne", "Xia", "Yara", "Zeke", "Aria", "Bruno",
    "Cassia", "Dante", "Elara", "Finley", "Gideon", "Helena", "Ivan", "Jolie", "Keiran", "Layla",
    "Malik", "Nova", "Oriana", "Porter", "Rhea", "Stefan", "Toby", "Vivian", "Weston", "Zora"
]

In [ ]:
def coin_flip_outcome_normal(start: str, actions: list[str]) -> str:
    flips = actions.count("flip")
    is_heads = (start == "heads")
    if flips % 2 == 1:
        is_heads = not is_heads
    return 'heads' if is_heads else 'tails'

def coin_flip_outcome_counter(start: str, actions: list[str]) -> str:
    flips = actions.count("flip")
    is_heads = (start == "heads")
    if flips % 2 == 1:
        is_heads = not is_heads
    return 'tails' if is_heads else 'heads'

def generate_coin_flip(train_path, val_path, total=5000, val_ratio=0.1, num_people=4, type='normal'):
    random.seed(42)
    used_train_keys = set()
    used_val_keys = set()
    
    train_size = int(total * (1 - val_ratio))
    val_size = total - train_size

    shuffled_names = ALL_NAMES[:]
    random.shuffle(shuffled_names)
    split_index = int(len(shuffled_names) * (1 - val_ratio))
    train_names_pool = shuffled_names[:split_index]
    val_names_pool = shuffled_names[split_index:]

    os.makedirs(os.path.dirname(train_path), exist_ok=True)
    os.makedirs(os.path.dirname(val_path), exist_ok=True)

    with open(train_path, "w", newline='') as f_train, open(val_path, "w", newline='') as f_val:
        train_writer = csv.writer(f_train)
        val_writer = csv.writer(f_val)
        train_writer.writerow(["clean", "corrupted", "answer"])
        val_writer.writerow(["clean", "corrupted", "answer"])

        train_written = 0
        val_written = 0

        # Generate train set
        while train_written < train_size:
            names = random.sample(train_names_pool, num_people)
            actions = [random.choice(["flip", "not flip"]) for _ in names]
            start = random.choice(["heads", "tails"])
            key = (start, tuple(names), tuple(actions))
            if key in used_train_keys:
                continue
            used_train_keys.add(key)

            steps = [f"{n} {'flips' if a == 'flip' else "doesn't"}." for n, a in zip(names, actions)]

            clean_prompt = f"A coin is {start} up. {' '.join(steps)} The coin is"
            if type == 'normal':
                answer = coin_flip_outcome_normal(start, actions)

            elif type == 'counter':
                answer = coin_flip_outcome_counter(start, actions)

            corrupted_start = "tails" if start == "heads" else "heads"
            corrupted_prompt = f"A coin is {corrupted_start} up. {' '.join(steps)} The coin is"

            train_writer.writerow([clean_prompt, corrupted_prompt, answer])
            train_written += 1

        while val_written < val_size:
            names = random.sample(val_names_pool, num_people)
            actions = [random.choice(["flip", "not flip"]) for _ in names]
            start = random.choice(["heads", "tails"])
            key = (start, tuple(names), tuple(actions))
            if key in used_val_keys:
                continue
            used_val_keys.add(key)

            steps = [f"{n} {'flips' if a == 'flip' else "doesn't"}." for n, a in zip(names, actions)]

            clean_prompt = f"A coin is {start} up. {' '.join(steps)} The coin is"
            
            if type == 'normal':
                answer = coin_flip_outcome_normal(start, actions)

            elif type == 'counter':
                answer = coin_flip_outcome_counter(start, actions)

            corrupted_start = "tails" if start == "heads" else "heads"
            corrupted_prompt = f"A coin is {corrupted_start} up. {' '.join(steps)} The coin is"

            val_writer.writerow([clean_prompt, corrupted_prompt, answer])
            val_written += 1

    print(f"✅ Train and validation CSVs saved to:\n - {train_path}\n - {val_path}")


In [19]:
generate_coin_flip(
    train_path="coin_flip/datasets_csv/train.csv",
    val_path="coin_flip/datasets_csv/validation.csv",
    total=6000,
    val_ratio=0.1,
    type = 'normal'
)

✅ Train and validation CSVs saved to:
 - coin_flip/datasets_csv/train.csv
 - coin_flip/datasets_csv/validation.csv


In [22]:
generate_coin_flip(
    train_path="coin_flip_inverse/datasets_csv/train.csv",
    val_path="coin_flip_inverse/datasets_csv/validation.csv",
    total=6000,
    val_ratio=0.1,
    type = 'counter'
)

✅ Train and validation CSVs saved to:
 - coin_flip_inverse/datasets_csv/train.csv
 - coin_flip_inverse/datasets_csv/validation.csv


In [9]:
import os
import csv
import random

def coin_flip_outcome_normal(start: str, actions: list[str]) -> str:
    flips = actions.count("flip")
    is_heads = (start == "heads")
    if flips % 2 == 1:
        is_heads = not is_heads
    return 'heads' if is_heads else 'tails'

def coin_flip_outcome_counter(start: str, actions: list[str]) -> str:
    flips = actions.count("flip")
    is_heads = (start == "heads")
    if flips % 2 == 1:
        is_heads = not is_heads
    return 'tails' if is_heads else 'heads'

def coin_flip_outcome_cycle(start: str, actions: list[str]) -> str:
    assert start in ["heads", "tails"], "Start cannot be edge"
    cycle = ["heads", "edge", "tails", "edge"]
    idx = cycle.index(start)
    for action in actions:
        if action == "flip":
            idx = (idx + 1) % 4
        # 'not flip' does nothing
    return cycle[idx]

def generate_coin_flip(train_path, val_path, total=5000, val_ratio=0.1, num_people=4, type='normal'):
    random.seed(42)
    used_train_keys = set()
    used_val_keys = set()
    
    train_size = int(total * (1 - val_ratio))
    val_size = total - train_size

    shuffled_names = ALL_NAMES[:]
    random.shuffle(shuffled_names)
    split_index = int(len(shuffled_names) * (1 - val_ratio))
    train_names_pool = shuffled_names[:split_index]
    val_names_pool = shuffled_names[split_index:]

    os.makedirs(os.path.dirname(train_path), exist_ok=True)
    os.makedirs(os.path.dirname(val_path), exist_ok=True)

    def determine_answer(start, actions):
        if type == 'normal':
            return coin_flip_outcome_normal(start, actions)
        elif type == 'counter':
            return coin_flip_outcome_counter(start, actions)
        elif type == 'cycle':
            return coin_flip_outcome_cycle(start, actions)
        else:
            raise ValueError("Invalid type")

    with open(train_path, "w", newline='') as f_train, open(val_path, "w", newline='') as f_val:
        train_writer = csv.writer(f_train)
        val_writer = csv.writer(f_val)
        train_writer.writerow(["clean", "corrupted", "answer"])
        val_writer.writerow(["clean", "corrupted", "answer"])

        def generate_set(pool, writer, used_keys, max_count):
            written = 0
            while written < max_count:
                names = random.sample(pool, num_people)
                actions = [random.choice(["flip", "not flip"]) for _ in names]
                start = random.choice(["heads", "tails"])  # No "edge" start
                key = (start, tuple(names), tuple(actions))
                if key in used_keys:
                    continue
                used_keys.add(key)

                steps = [f"{n} {'flips' if a == 'flip' else 'does not flip'}." for n, a in zip(names, actions)]
                clean_prompt = f"A coin is {start} up. {' '.join(steps)} The coin is"
                answer = determine_answer(start, actions)

                corrupted_start = "tails" if start == "heads" else "heads"
                corrupted_prompt = f"A coin is {corrupted_start} up. {' '.join(steps)} The coin is"

                writer.writerow([clean_prompt, corrupted_prompt, answer])
                written += 1

        generate_set(train_names_pool, train_writer, used_train_keys, train_size)
        generate_set(val_names_pool, val_writer, used_val_keys, val_size)

    print(f"✅ Train and validation CSVs saved to:\n - {train_path}\n - {val_path}")


In [10]:
generate_coin_flip(
    train_path="three_sided_coin/datasets_csv/train.csv",
    val_path="three_sided_coin/datasets_csv/validation.csv",
    total=6000,
    val_ratio=0.1,
    num_people=4,
    type='cycle'
)

✅ Train and validation CSVs saved to:
 - three_sided_coin/datasets_csv/train.csv
 - three_sided_coin/datasets_csv/validation.csv
